# 1.5 Adding lower bounds to the model

In [ ]:
# install dependencies
%pip install -q amplpy pandas

from amplpy import AMPL, ampl_notebook
import pandas as pd

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics

Once the model and data have been set up, it is a simple matter to
change them and then re-solve.  Indeed, we would not expect to find
an LP application in which the model and data are prepared and
solved just once, or even a few times.  Most commonly, numerous
refinements are introduced as the model is developed, and changes
to the data continue for as long as the model is used.

Let's conclude this chapter with a few examples of changes and
refinements.  These examples also highlight some additional
features of AMPL.

Suppose first that we add another product, steel plate.  The model
stays the same, but in the data we have to add
`plate`
to the list of members for the set
`PROD` ,
and we have to add a line of parameter values for
`plate` :

We put this version of the data in a [Pandas](https://pandas.pydata.org/) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html),
and use AMPL as before to get the solution:

In [2]:
df_steel = pd.DataFrame(
    [
        ['bands', 200, 25, 6000],
        ['coils', 140, 30, 4000],
        ['plate', 160, 29, 3500]
    ],
    columns=['PROD', 'rate', 'profit', 'market']
).set_index("PROD")
avail = 40

# instantiate AMPL object
ampl = AMPL()
# read the model
ampl.read('steel.mod')
# read the data
ampl.set_data(df_steel, 'PROD')
ampl.param['avail'] = avail
# choose solver and solve
ampl.solve(solver='highs')
# display results for the 'Make' variable
ampl.display('Make')

HiGHS 1.6.0:HiGHS 1.6.0: optimal solution; objective 196400
1 simplex iterations
0 barrier iterations
 
Make [*] :=
bands  6000
coils     0
plate  1600
;



Profits have increased compared to the two-variable version, but
now it is best to produce no coils at all!  On closer examination, this
result is not so surprising.  Plate yields a profit of \$4640 per
hour, which is less than for bands but more than for coils.  Thus
plate is produced to absorb the capacity not taken by bands; coils
would be produced only if both bands and plate reached their market
limits before the available hours were exhausted.

In reality, a whole product line cannot be shut
down solely to increase weekly profits.  The simplest way to reflect
this in the model is to add lower bounds on the production amounts,
as shown in Figures [1-5a](#fig-1-5a) and
[1-5b](#fig-1-5b).
We have declared a new collection of parameters named
`commit` ,
to represent the lower bounds on production that are imposed by
sales commitments, and we have changed
`>= 0`
to
`>= commit[p]`
in the declaration of the variables
`Make[p]`.

<a id='fig-1-5a'><center><b>Figure 1-5a:</b> Lower bounds on production (steel3.mod).</center></a>

In [3]:
%%writefile steel3.mod

set PROD;  # products

param rate {PROD} > 0;     # produced tons per hour
param avail >= 0;          # hours available in week
param profit {PROD};       # profit per ton

param commit {PROD} >= 0;  # lower limit on tons sold in week
param market {PROD} >= 0;  # upper limit on tons sold in week

var Make {p in PROD} >= commit[p], <= market[p]; # tons produced

# Objective: total profits from all products
maximize Total_Profit: sum {p in PROD} profit[p] * Make[p];
               
# Constraint: total of hours used by all
# products may not exceed hours available
subject to Time: sum {p in PROD} (1/rate[p]) * Make[p] <= avail;               

Overwriting steel3.mod


<a id='fig-1-5b'><center><b>Figure 1-5b:</b> Data for lower bounds on production.</center></a>

In [4]:
df_steel3 = pd.DataFrame(
    [
        ['bands', 200, 25, 1000, 6000],
        ['coils', 140, 30, 500, 4000],
        ['plate', 160, 29, 750, 3500]
    ],
    columns=['PROD', 'rate', 'profit', 'commit', 'market']
).set_index("PROD")
avail = 40

After these changes are made, we can run AMPL again to get a more
realistic solution:

In [5]:
# instantiate AMPL object
ampl = AMPL()
# read the model
ampl.read('steel3.mod')
# read the data
ampl.set_data(df_steel3, 'PROD')
ampl.param['avail'] = avail
# choose solver and solve
ampl.solve(solver='highs')
# display results for the 'Make' variable
ampl.display('commit', 'Make', 'market')

HiGHS 1.6.0:HiGHS 1.6.0: optimal solution; objective 194828.5714
1 simplex iterations
0 barrier iterations
 
:     commit    Make   market    :=
bands   1000   6000      6000
coils    500    500      4000
plate    750   1028.57   3500
;



For comparison, we have displayed
`commit`
and
`market`
on either side of the actual production,
`Make` .
As expected, after the commitments are met, it is most profitable to
produce bands up to the market limit, and then to produce plate with
the remaining available time.